# Descriptive analysis for the manuscript

Summarize geotagged tweets of the multiple regions used for the experiment and the application.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import numpy as np
import pandas as pd
import yaml
import scipy.stats as stats
from  tqdm import tqdm

def load_region_tweets(region=None):
    df = pd.read_csv(f'../../dbs/{region}/geotweets.csv')
    df['day'] = df['createdat'].apply(lambda x: x.split(' ')[0])
    df['createdat'] = pd.to_datetime(df['createdat'], infer_datetime_format=True)
    t_max, t_min = df.createdat.max(), df.createdat.min()
    time_span = f'{t_min} - {t_max}'
    num_users = len(df.userid.unique())
    num_geo = len(df)
    num_days = np.median(df.groupby(['userid'])['day'].nunique())
    num_geo_freq = np.median(df.groupby(['userid']).size() / df.groupby(['userid'])['day'].nunique())
    return region, time_span, num_users, num_geo, num_days, num_geo_freq

def user_stats_cal(data):
    time_span = data.createdat.max() - data.createdat.min()
    time_span = time_span.days
    if time_span == 0:
        time_span += 1
    num_days = data['day'].nunique()
    num_geo = len(data)
    geo_freq = num_geo / num_days
    share_active = num_days / time_span
    return pd.DataFrame.from_dict({'time_span': [time_span],
            'num_days': [num_days],
            'num_geo': [num_geo],
            'geo_freq': [geo_freq],
            'share_active': [share_active]
            })

def region_tweets_stats_per_user(region=None):
    df = pd.read_csv(f'../../dbs/{region}/geotweets.csv')
    df['day'] = df['createdat'].apply(lambda x: x.split(' ')[0])
    df['createdat'] = pd.to_datetime(df['createdat'], infer_datetime_format=True)
    tqdm.pandas(desc=region)
    df_users = df.groupby('userid').progress_apply(user_stats_cal).reset_index()
    df_users.loc[:, 'region'] = region
    df_users.drop(columns=['level_1'], inplace=True)
    return df_users

region_list = ['sweden', 'netherlands', 'saopaulo', 'australia', 'austria', 'barcelona',
               'capetown', 'cebu', 'egypt', 'guadalajara', 'jakarta',
               'johannesburg', 'kualalumpur', 'lagos', 'madrid', 'manila', 'mexicocity', 'moscow', 'nairobi',
               'rio', 'saudiarabia', 'stpertersburg', 'surabaya']

with open('../../lib/regions.yaml', encoding='utf8') as f:
    region_manager = yaml.load(f, Loader=yaml.FullLoader)

## 1 Summarize the geotagged tweets used as input to the model
Geotagged tweets: Time span, No. of Twitter users, No. of geotagged tweets,
Days covered/user, No. of geotagged tweets/day/user

In [6]:
df = pd.DataFrame([load_region_tweets(region=x) for x in region_list],
                  columns=('region', 'time_span', 'num_users', 'num_geo', 'num_days', 'num_geo_freq'))
df.loc[:, 'gdp_capita'] = df.loc[:, 'region'].apply(lambda x: region_manager[x]['gdp_capita'])
df.loc[:, 'country'] = df.loc[:, 'region'].apply(lambda x: region_manager[x]['country'])
df.loc[:, 'pop'] = df.loc[:, 'region'].apply(lambda x: region_manager[x]['pop'])
df.loc[:, 'time_span'] = df.loc[:, 'time_span'].apply(lambda x: ' - '.join([x_t.split(' ')[0] for x_t in x.split(' - ')]))
df.loc[:, 'region'] = df.loc[:, 'region'].apply(lambda x: region_manager[x]['name'])
df

KeyboardInterrupt: 

In [ ]:
df.to_clipboard(index=False)

## 1-extra Summarize the geotagged tweets used as input to the model - by user
This is for dissertation presentation - sparsity issue.

Geotagged tweets: Time span, No. of Twitter users, No. of geotagged tweets,
Days covered/user, No. of geotagged tweets/day/user

In [25]:
df = pd.concat([region_tweets_stats_per_user(region=x) for x in region_list])
df.loc[:, 'gdp_capita'] = df.loc[:, 'region'].apply(lambda x: region_manager[x]['gdp_capita'])
df.loc[:, 'country'] = df.loc[:, 'region'].apply(lambda x: region_manager[x]['country'])
df.loc[:, 'pop'] = df.loc[:, 'region'].apply(lambda x: region_manager[x]['pop'])
df.loc[:, 'region'] = df.loc[:, 'region'].apply(lambda x: region_manager[x]['name'])
df.to_csv(f'../../dbs/regional_stats.csv', index=False)

surabaya: 100%|██████████| 2414/2414 [00:02<00:00, 807.53it/s]


## 2 Merge ODMs for visualisation
This part applies to Sweden, The Netherlands, and Sao Paulo, Brazil.

Separate files will be deleted.

In [4]:
for region in ['sweden', 'netherlands', 'saopaulo']:
    df = pd.read_csv(f'../../dbs/{region}/odm_gt.csv')
    df_c = pd.read_csv(f'../../dbs/{region}/odm_calibration.csv')
    df_v = pd.read_csv(f'../../dbs/{region}/odm_validation.csv')
    df_cb = pd.read_csv(f'../../dbs/{region}/odm_benchmark_c.csv')
    df_vb = pd.read_csv(f'../../dbs/{region}/odm_benchmark_v.csv')
    df = pd.merge(df, df_c, on=['ozone', 'dzone'])
    df = df.rename(columns={'model': 'model_c'})
    df = pd.merge(df, df_v, on=['ozone', 'dzone'])
    df = df.rename(columns={'model': 'model_v'})
    df = pd.merge(df, df_cb, on=['ozone', 'dzone'])
    df = df.rename(columns={'benchmark': 'benchmark_c'})
    df = pd.merge(df, df_vb, on=['ozone', 'dzone'])
    df = df.rename(columns={'benchmark': 'benchmark_v'})
    df.loc[:, ['ozone', 'dzone',
               'gt', 'model_c', 'model_v',
               'benchmark_c', 'benchmark_v']].to_csv(f'../../dbs/{region}/odms.csv', index=False)
    os.remove(f'../../dbs/{region}/odm_gt.csv')
    os.remove(f'../../dbs/{region}/odm_calibration.csv')
    os.remove(f'../../dbs/{region}/odm_validation.csv')
    os.remove(f'../../dbs/{region}/odm_benchmark_c.csv')
    os.remove(f'../../dbs/{region}/odm_benchmark_v.csv')

## 3 Quantify the od-pair similarity
This part applies to Sweden, The Netherlands, and Sao Paulo, Brazil.

The overall similarity.

In [9]:
quant_list = []
for region in ['sweden', 'netherlands', 'saopaulo']:
    df = pd.read_csv(f'../../dbs/{region}/odms.csv')
    df_c = df.loc[(df.gt != 0) & (df.model_c != 0) & (df.benchmark_c != 0), :]
    mc = stats.kendalltau(df_c.loc[:, 'gt'], df_c.loc[:, 'model_c'])
    quant_list.append((region, 'model', 'c', mc.correlation, mc.pvalue))

    bc = stats.kendalltau(df_c.loc[:, 'gt'], df_c.loc[:, 'benchmark_c'])
    quant_list.append((region, 'benchmark', 'c', bc.correlation, bc.pvalue))

    df_v = df.loc[(df.gt != 0) & (df.model_v != 0) & (df.benchmark_v != 0), :]
    mv = stats.kendalltau(df_v.loc[:, 'gt'], df_v.loc[:, 'model_v'])
    quant_list.append((region, 'model', 'v', mv.correlation, mv.pvalue))

    bv = stats.kendalltau(df_v.loc[:, 'gt'], df_v.loc[:, 'benchmark_v'])
    quant_list.append((region, 'benchmark', 'v', bv.correlation, bv.pvalue))
df_stats = pd.DataFrame(quant_list, columns=['region', 'type', 'data', 'cor', 'p'])
df_stats

,region,type,data,cor,p
0,sweden,model,c,0.187270,6.653740e-246
1,sweden,benchmark,c,0.220507,9.577697e-277
2,sweden,model,v,0.184070,2.009274e-128
3,sweden,benchmark,v,0.266851,3.416318e-215
4,netherlands,model,c,0.439008,0.000000e+00
5,netherlands,benchmark,c,0.331307,0.000000e+00
6,netherlands,model,v,0.432798,0.000000e+00
7,netherlands,benchmark,v,0.404650,0.000000e+00
8,saopaulo,model,c,0.414933,0.000000e+00
9,saopaulo,benchmark,c,0.296695,0.000000e+00


In [10]:
df_stats.groupby(['region', 'type'])['cor'].mean()

region       type     
netherlands  benchmark    0.367978
             model        0.435903
saopaulo     benchmark    0.326389
             model        0.447656
sweden       benchmark    0.243679
             model        0.185670
Name: cor, dtype: float64

## 4 Check the total PKT of The Netherlands using Survey

In [ ]:
sheet1 = pd.read_excel("../../dbs/netherlands/mobility_data/OViN2017_Databestand.xlsx")
trips = sheet1.loc[:, ['OPID', 'AfstV', 'FactorV']]
trips = trips.rename(columns={
    'OPID': 'userid',
    'AfstV': 'distance',
    'FactorV': 'weight_trip'
})
# Prepare the actual trip distances
trips_d = trips.dropna(subset=['distance'])
trips_d.loc[:, 'distance'] = trips_d.loc[:, 'distance'] / 10 # hectometer to km

In [ ]:
trips_capita = pd.DataFrame(trips_d.groupby('userid')['distance'].sum() * 365 / 1000,
                            columns=['distance']).reset_index() # 1000 km
trips_capita = pd.merge(trips_capita, trips_d.loc[:, ['userid', 'weight_trip']].drop_duplicates(subset=['weight_trip']), on='userid')
trips_capita.head()

In [ ]:
import weightedstats as ws
ws.numpy_weighted_median(trips_capita['distance'], weights=trips_capita['weight_trip']), \
np.average(trips_capita['distance'], weights=trips_capita['weight_trip'])

## 5-extra Temporal profile of geotagged tweets
Sweden

In [36]:
region = 'sweden'
time_list = [(w, h) for w in range(0, 7) for h in range(0, 24)]
time_dict = {x: y for x,y in zip(time_list, range(0, len(time_list)))}
df_gt = pd.read_csv(f'../../dbs/{region}/survey_deso/desti_temporal_survey.csv')
df_gt['count'] = df_gt['count'] / df_gt['count'].sum() * 100
df_gt.loc[:, 'seq'] = df_gt.apply(lambda row: time_dict[(row['weekday'], row['hour'])], axis=1)
df_gt.loc[:, 'source'] = 'Travel survey'

df_tw = pd.read_csv(f'../../dbs/{region}/geotweets.csv')
df_tw['createdat'] = pd.to_datetime(df_tw['createdat'], infer_datetime_format=True)
df_tw['weekday'] = df_tw['createdat'].apply(lambda x: x.weekday())
df_tw['hour'] = df_tw['createdat'].apply(lambda x: x.hour)
df_tw_tempo = df_tw.groupby(['weekday', 'hour']).size().reset_index()
df_tw_tempo.rename(columns={0: 'count'}, inplace=True)
df_tw_tempo['count'] = df_tw_tempo['count'] / df_tw_tempo['count'].sum() * 100
df_tw_tempo.loc[:, 'seq'] = df_tw_tempo.apply(lambda row: time_dict[(row['weekday'], row['hour'])], axis=1)
df_tw_tempo.loc[:, 'source'] = 'Geotagged tweets'

In [37]:
pd.concat([df_tw_tempo, df_gt]).to_csv(f'../../dbs/{region}/temporal_tw_vs_survey.csv', index=False)